In [1]:
import sys
import numpy as np
import os, h5py
import pandas as pd
from gopher import variant_effect

2023-04-25 13:23:18.624157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 13:23:19.173400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# read df and add strand
all_dfs = []
cagi_data = '../../data/CAGI/raw/'

for filename in os.listdir(cagi_data):
    prefix, regulator = filename.split('.tsv')[0].split('_')

    one_reg = pd.read_csv(os.path.join(cagi_data,filename), skiprows=7, sep='\t', header=None)
    one_reg['regulator'] = regulator
    one_reg['set'] = prefix
    all_dfs.append(one_reg)

combined_cagi = pd.concat(all_dfs)
combined_cagi.insert(4, 'strand', '+')
combined_cagi.insert(2,'end',combined_cagi.iloc[:,1]+1)
combined_cagi.iloc[:,0] = 'chr'+combined_cagi.iloc[:,0].astype(str)


In [12]:
K562_file = '../../data/CAGI/K562_cagi.bed'
K562_cagi = combined_cagi[combined_cagi['regulator']=='PKLR']
K562_cagi.to_csv(K562_file, sep='\t', header=False, index=None)
hepg2_file = '../../data/CAGI/HepG2_cagi.bed'
exp_list = ['F9','LDLR','SORT1']
hepg2_cagi = combined_cagi[combined_cagi['regulator'].isin(exp_list)]
hepg2_cagi.to_csv(hepg2_file, sep='\t', header=False, index=None)

In [14]:
output_filename = '../../data/CAGI/cagi_hepg2_230.bed'
variant_effect.expand_range(hepg2_file, output_filename,window=230)
output_filename = '../../data/CAGI/cagi_k562_230.bed'
variant_effect.expand_range(K562_file, output_filename,window=230)

In [15]:
!bedtools getfasta -fi /home/ztang/ref/hg19/hg19.fa -bed /home/ztang/multitask_RNA/data/CAGI/cagi_hepg2_230.bed -fo /home/ztang/multitask_RNA/data/CAGI/cagi_hepg2.fa
!bedtools getfasta -fi /home/ztang/ref/hg19/hg19.fa -bed /home/ztang/multitask_RNA/data/CAGI/cagi_k562_230.bed -fo /home/ztang/multitask_RNA/data/CAGI/cagi_k562.fa

In [33]:
for celltype in ['k562','hepg2']:
    fa_filename = '../../data/CAGI/cagi_'+celltype+'.fa'
    output_filename = '../../data/CAGI/cagi_'+celltype+'_230.bed'
    coords_list, seqs_list = variant_effect.convert_bed_to_seq(output_filename, fa_filename, genomefile='/home/ztang/ref/hg19/hg19.fa')
    window = 230
    bad_lines = []
    N = len(seqs_list)
    nonneg_df = pd.read_csv(output_filename, sep='\t', header=None)
    mid = window // 2
    onehot_ref = []
    onehot_alt = []
    coord_np = np.empty((N, 4)) # chrom, start, end coordinate array
    pos_dict = {'+': 114, '-':115}
    for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
        alt = ''
        strand = chr_s_e.split('(')[-1].split(')')[0]
        pos = pos_dict[strand]
    #     coord_np[i,3] = pos_dict[strand] - 1535

        if seq[pos] != nonneg_df.iloc[i, 3]:
    #         print('Error in line ' + str(i))
            bad_lines.append(i)
        else:
            alt = nonneg_df.iloc[i,4]

            onehot = variant_effect.dna_one_hot(seq)
            mutated_onehot = onehot.copy()
            mutated_onehot[pos] = variant_effect.dna_one_hot(alt)[0]
            onehot_ref.append(onehot)

            onehot_alt.append(mutated_onehot) 

    onehot_alt = np.array(onehot_alt)
    onehot_ref = np.array(onehot_ref)

    included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
    included_df.to_csv('../../data/CAGI/'+celltype+'_metadata.csv')

    onehot_ref_alt = h5py.File('../../data/CAGI/'+celltype+'_onehot.h5', 'w')
    onehot_ref_alt.create_dataset('ref', data=onehot_ref)
    onehot_ref_alt.create_dataset('alt', data=onehot_alt)
    onehot_ref_alt.close()

In [34]:
bad_lines

[1385, 1386, 1387, 2056, 2057]

In [39]:
seqs_list[2056][1]

'G'

In [38]:
nonneg_df.iloc[2056]

0         chr1
1    109817158
2    109817388
3            C
4            A
5            +
6        -0.23
7         0.02
8        SORT1
9    challenge
Name: 2056, dtype: object